# 闭包Closure
## 闭包的写法
在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，返回内部函数

In [1]:
# 定义一个函数
def test(number):
    # 在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么将这个函数以及用到的一些变量称之为闭包
    def test_in(number_in):
        print("in test_in 函数, number_in is %d" % number_in)
        return number+number_in
    # 其实这里返回的就是闭包的结果
    return test_in


# 给test 函数赋值，这个20 就是给参数number
ret = test(20)
# 注意这里的100 其实给参数number_in
print(ret(100))


in test_in 函数, number_in is 100
120


## nonlocal与global的区别

In [1]:
def counter(start=0):
    def incr():
        nonlocal start # nonlocal关键字声明变量为外层函数的变量
        # q:nonlocal与global的区别
        # a:nonlocal关键字声明变量为外层函数的变量，global关键字声明变量为全局变量
        start += 1
        return start
    return incr


c1 = counter(5)
print(c1())
print(c1())
c2 = counter(50)
print(c2())
print(c2())
print(c1())
print(c1())
print(c2())
print(c2())


6
7
51
52
8
9
53
54


In [3]:
def counter(start=0):
    def incr():
        global start # global关键字声明变量为全局变量
        start += 1
        return start
    return incr

start = 1
c1 = counter(5)
print(c1())
print(c1())
c2 = counter(50)
print(c2())
print(c2())
print(c1())
print(c1())
print(c2())
print(c2())


2
3
4
5
6
7
8
9


# 装饰器
## 装饰器执行的顺序
装饰是由内而外进行装饰，离函数近的先装饰；执行是由外而内，离函数远的先装饰，类似于栈的操作

In [7]:
# 多个装饰器装饰同一个函数
def add_first(func):
    print("---开始进行装饰权限1 的功能---")

    def call_func(*args, **kwargs):
        print("---这是权限验证1----")
        return func(*args, **kwargs)
    return call_func


def add_second(func):
    print("---开始进行装饰权限2 的功能---")

    def call_func(*args, **kwargs):
        print("---这是权限验证2----")
        return func(*args, **kwargs)
    return call_func

def add_third(func):
    print("---开始进行装饰权限3 的功能---")

    def call_func(*args, **kwargs):
        print("---这是权限验证3----")
        return func(*args, **kwargs)
    return call_func

@add_first # 最后装饰，最先执行
@add_second 
@add_third # 最先装饰，最后执行
def test1(a):
    print("------test1------", a)


test1(1)


---开始进行装饰权限3 的功能---
---开始进行装饰权限2 的功能---
---开始进行装饰权限1 的功能---
---这是权限验证1----
---这是权限验证2----
---这是权限验证3----
------test1------ 1


## 装饰器有return

In [4]:
from time import ctime, sleep


def timefun(func):
    def wrapped_func():
        print("%s called at %s" % (func.__name__, ctime()))
        # func() # 直接执行func()，ret为None
        return func() # 返回func()，ret为func()的返回值

    return wrapped_func # ret为None


@timefun
def foo():
    print("I am foo")


@timefun
def getInfo():
    return '----hahah---'


foo()
sleep(1)
foo()
ret = getInfo()
print(ret)


foo called at Fri Mar 24 10:51:35 2023
I am foo
foo called at Fri Mar 24 10:51:36 2023
I am foo
getInfo called at Fri Mar 24 10:51:36 2023
----hahah---


## 装饰器带参数
在原有装饰器的基础上，设置外部变量

In [6]:
from time import ctime, sleep


def timefun_arg(pre="hello"):
    def timefun(func):
        def wrapped_func():
            print("%s called at %s %s" % (func.__name__, ctime(), pre))
            return func()
        return wrapped_func
    return timefun
# 下面的装饰过程
# 1. 调用timefun_arg("wangdao")
# 2. 将步骤1 得到的返回值，即time_fun 返回， 然后time_fun(foo)
# 3. 将time_fun(foo)的结果返回，即wrapped_func
# 4. 让foo = wrapped_fun，即foo 现在指向wrapped_func


@timefun_arg("wangdao")
def foo():
    print("I am foo")


@timefun_arg("python")
def too():
    print("I am too")


foo()
sleep(2)
foo()
too()
sleep(2)
too()


foo called at Fri Mar 24 10:54:25 2023 wangdao
I am foo
foo called at Fri Mar 24 10:54:27 2023 wangdao
I am foo
too called at Fri Mar 24 10:54:27 2023 python
I am too
too called at Fri Mar 24 10:54:29 2023 python
I am too


## 类装饰器

In [10]:
class Test(object):
    def __init__(self, func):
        print("---初始化---")
        print("func name is %s" % func.__name__)
        self.__func = func

    def __call__(self):
        print("---装饰器中的功能---")
        self.__func()


@Test  # 相当于test = Test(test)
def test():
    print("----test---")


test()  # 注释掉依然会看到："--初始化--" func name is test


---初始化---
func name is test
---装饰器中的功能---
----test---


## 增加warps 的作用
warps 保留原函数的自己本身的名字和文档

In [13]:
from functools import wraps
def my_decorator(func):
    def wper(*args, **kwargs):
        '''decorator'''
        print('Calling decorated function...')
        return func(*args, **kwargs)
    return wper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

print(example.__name__, "\t\t\t", example.__doc__)#函数名和文档字符串已经被覆盖


wper 			 decorator


In [14]:
from functools import wraps


def my_decorator(func):
    @wraps(func) # 保留原函数的元信息
    def wper(*args, **kwargs):
        '''decorator'''
        print('Calling decorated function...')
        return func(*args, **kwargs)
    return wper


@my_decorator
def example():
    """Docstring"""
    print('Called example function')


print(example.__name__, "\t\t\t", example.__doc__)  # 函数名和文档字符串还是原来的


example 			 Docstring
